<a href="https://colab.research.google.com/github/safitrisoetam/Bangkit-Capstone-Project/blob/main/notebook/IndoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirement

In [ ]:
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
import os
import numpy as np
import pandas as pd

Mounted at /content/drive


In [ ]:
!pip install transformers
import re
import string
import torch
from torch import clamp
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 880 kB 48.2 MB/s 
     |████████████████████████████████| 596 kB 58.5 MB/s 
     |████████████████████████████████| 6.6 MB 58.0 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=72468fdbfd273690565959ff4ecee0b7549dd17153e09ad68cc6315fab737fd5
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Import Dataset

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Safitri/Phase_5/dataupdated.csv')
data.info()
data.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      487 non-null    object
 1   web-scraper-start-url  487 non-null    object
 2   pagelink               487 non-null    object
 3   pagelink-href          487 non-null    object
 4   judul                  487 non-null    object
 5   penulis                487 non-null    object
 6   jurnal                 487 non-null    object
 7   penerbit               487 non-null    object
 8   tanggal                487 non-null    object
 9   abstrak                487 non-null    object
 10  judul-stemmed          487 non-null    object
 11  abstrak-stemmed        487 non-null    object
 12  judul-stopped          487 non-null    object
 13  abstrak-stopped        487 non-null    object
 14  konten                 487 non-null    object
 15  js-count               

,web-scraper-order,web-scraper-start-url,pagelink,pagelink-href,judul,penulis,jurnal,penerbit,tanggal,abstrak,judul-stemmed,abstrak-stemmed,judul-stopped,abstrak-stopped,konten,js-count,year
0,1642827697-57,https://garuda.kemdikbud.go.id/journal/view/12...,Model Matematika dan Analisanya Dari Pemenuhan...,https://garuda.kemdikbud.go.id/documents/detai...,model matematika dan analisanya dari pemenuhan...,widodo basuki,Limits,institut teknologi sepuluh nopember surabaya,2004-05-26,air bersih merupakan kebutuhan manusia yang me...,model matematika dan analisanya dari penuh but...,air bersih rupa butuh manusia yang dapat perha...,model matematika analisanya penuh butuh air be...,air bersih rupa butuh manusia dapat perhati kh...,model matematika analisanya penuh butuh air be...,10,2004


# Method 1

*   [Indonesian (IndoBERT) Tutorial](https://https://utomorezadwi.medium.com/model-text-similarity-menggunakan-bert-berbasis-bahasa-indonesia-14898e301bef)
*   [English Step-by-Step Tutorial](https://https://utomorezadwi.medium.com/model-text-similarity-menggunakan-bert-berbasis-bahasa-indonesia-14898e301bef)

## Dataframe to List
For ease of access

In [ ]:
df_sentences = ['aljabar','analisis','citra','data','industri','keuangan','pemodelan',
                'pemrograman','pendidikan','simulasi']
len(df_sentences)

10

In [ ]:
df = data['konten']
print(df)
df = df.tolist()
len(df)

0      model matematika analisanya penuh butuh air be...
1      desain kontroler tunggal redam osilasi multi f...
2      studi komparatif jaring syaraf tiru boltzman m...
3      terap matriks transisi ubah fungsi interpolasi...
4      regresi spline prediksi model hubung temperatu...
                             ...                        
482    rencana lokasi stasiun isi bahan bakar gas spb...
483    optimasi jalur evakuasi pejal kaki algoritma f...
484    analisis model matematika sebar sakit covid lo...
485    analisis asuransi anggur asumsi non zero morta...
486    model dinamika transmisi sakit schistosomiasis...
Name: konten, Length: 487, dtype: object


487

In [ ]:
df_sentences = df_sentences + df
print(df_sentences)
len(df_sentences)

['aljabar', 'analisis', 'citra', 'data', 'industri', 'keuangan', 'pemodelan', 'pemrograman', 'pendidikan', 'simulasi', 'model matematika analisanya penuh butuh air bersih suatu kompleks rumah air bersih rupa butuh manusia dapat perhati khusus penuh kompleks rumah ingin timbul pada penuh butuh air bersih hadap penuh butuh air bersih penuh kurang atau lama salah dapat debit kurang optimal pdam tuju kompleks rumah rumah yang terlalu kompleks pe rumah karena perlu model matematika selesai sebut dalam teliti ini guna kontinuitas dan sama bernoulli buat model matematika penuh butuh air bersih suatu kompleks rumah oleh hasil tekan air banding dengan waktu penuh', 'desain kontroler tunggal redam osilasi multi frekuensi sistem skala paper jelas tahap desain kontroler tunggal tuju redam dari frekuensi osilasi jadi suatu sistem skala masuk metoda metoda kembang tiap tahap metoda kembang utama guna desain kontroler sistem sangat serta milik multi frekuensi osilasi multi variabel state kandidat sin

497

## Class Breakdown

In [ ]:
#@title **def load_pretrained**
tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
model = AutoModel.from_pretrained('indobenchmark/indobert-base-p2')

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/224k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

In [ ]:
corpus = df_sentences
# initialize dictionary: stores tokenized sentences
token = {'input_ids': [], 'attention_mask': []}
for sentence in corpus:
    # encode each sentence, append to dictionary
    new_token = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    token['input_ids'].append(new_token['input_ids'][0])
    token['attention_mask'].append(new_token['attention_mask'][0])

token['input_ids'] = torch.stack(token['input_ids'])
token['attention_mask'] = torch.stack(token['attention_mask'])

output = model(**token)
embeddings = output.last_hidden_state
att_mask = token['attention_mask']
mask = att_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask_embeddings = embeddings * mask
summed = torch.sum(mask_embeddings, 1)
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
mean_pooled = summed / summed_mask

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()

In [ ]:
# calculate
similarity = []
similarity.append(cosine_similarity([mean_pooled[0]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[1]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[2]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[3]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[4]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[5]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[6]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[7]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[8]],mean_pooled[10:]))
similarity.append(cosine_similarity([mean_pooled[9]],mean_pooled[10:]))

In [ ]:
similarity

In [ ]:
score = pd.DataFrame(np.row_stack(similarity))
score = score.transpose()
score

## Define Class

In [ ]:
class TokenSimilarity:

    def load_pretrained(self, from_pretrained:str="indobenchmark/indobert-base-p1"):
        self.tokenizer = AutoTokenizer.from_pretrained(from_pretrained)
        self.model = AutoModel.from_pretrained(from_pretrained)

        
#    def __cleaning(self, text:str):
#        # clear punctuations
#        text = text.translate(str.maketrans('', '', string.punctuation))

        # clear multiple spaces
#        text = re.sub(r'/s+', ' ', text).strip()

#        return text
        
    def __process(self, first_token:str, second_token:str):
        inputs = self.tokenizer([first_token, second_token],
                                max_length=self.max_length,
                                truncation=self.truncation,
                                padding=self.padding,
                                return_tensors='pt')

        attention = inputs.attention_mask

        outputs = self.model(**inputs)

        # get the weights from the last layer as embeddings
        embeddings = outputs[0] # when used in older transformers version
        # embeddings = outputs.last_hidden_state # when used in newer one

        # add more dimension then expand tensor
        # to match embeddings shape by duplicating its values by rows
        mask = attention.unsqueeze(-1).expand(embeddings.shape).float()

        masked_embeddings = embeddings * mask
        
        # MEAN POOLING FOR 2ND DIMENSION
        # first, get sums by 2nd dimension
        # second, get counts of 2nd dimension
        # third, calculate the mean, i.e. sums/counts
        summed = masked_embeddings.sum(1)
        counts = clamp(mask.sum(1), min=1e-9)
        mean_pooled = summed/counts

        # return mean pooling as numpy array
        return mean_pooled.detach().numpy()
        
    def predict(self, first_token:str, second_token:str,
                return_as_embeddings:bool=False, max_length:int=16,
                truncation:bool=True, padding:str="max_length"):
        self.max_length = max_length
        self.truncation = truncation
        self.padding = padding

#        first_token = self.__cleaning(first_token)
#        second_token = self.__cleaning(second_token)

        mean_pooled_arr = self.__process(first_token, second_token)
        if return_as_embeddings:
            return mean_pooled_arr

        # calculate similarity
        similarity = cosine_similarity([mean_pooled_arr[0]], [mean_pooled_arr[1]])

        return similarity

### Download Model

In [ ]:
model = TokenSimilarity()
model.load_pretrained('indobenchmark/indobert-base-p2')

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/224k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

### Output

In [ ]:
kueri = 'aljabar'
judul1 = 'model matematika dan analisanya dari pemenuhan kebutuhan air bersih di suatu kompleks perumahanan'
judul2 = 'desain kontroler tunggal untuk meredam osilasi multi frekuensi pada sistem skala besar'

In [ ]:
print(model.predict(kueri, judul1))
print(model.predict(kueri, judul2))

[[0.25142747]]
[[0.23871209]]
